In [1]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from pathlib import Path
import numpy as np
import PIL
import PIL.Image
import json
from tqdm.auto import tqdm

#change working directory to root
ROOT_DIR = os.getcwd()
while os.path.basename(ROOT_DIR) != 'VisIrNet':
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR,'..'))
sys.path.insert(0,ROOT_DIR)
os.chdir(ROOT_DIR)

ROOT_DIR = Path(ROOT_DIR)

print(tf.__version__)
devices = tf.config.list_physical_devices('GPU')
print("len(devices): ", len(devices))
print(f"available GPUs: {devices}");


2.14.0
len(devices):  0
available GPUs: []


/cta/users/ndigande/miniconda3/envs/VisIrNet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## gpu setup 
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
###

**Configurations**

In [3]:
# config file to load will be passed as an argument
# get run parameters

import argparse 

parser = argparse.ArgumentParser()
parser.add_argument('--config-file', 
                        action = "store", 
                        dest = "config_file",
                        default = "vedai_default_config.json",
                        help = 'specify config file to load')

input_arguments = parser.parse_args([])

from Tools.configurations_parser import ConfigurationParser
# load configurations
configs = ConfigurationParser.getConfigurations(configs_path = 'configs', 
                                                config_file = str(input_arguments.config_file))


# print configurations
# ConfigurationParser.printConfigurations(configs)


[INFO] reading configurations from configs/vedai_default_config.json


**Dataloaders**

In [4]:
dataset_name = configs.dataset

DATASET_DIR = Path("data") / dataset_name 

split = "val"
input_images_dir = f"{split}_input"
template_images_dir=f"{split}_template"
labels_dir=f"{split}_label"

INPUTS_DIR = DATASET_DIR / input_images_dir
TEMPLATES_DIR = DATASET_DIR / template_images_dir
LABELS_DIR = DATASET_DIR / labels_dir    

In [5]:
print(f"[INFO] loading {split} dataset")
dataset = tf.data.Dataset.list_files(str(INPUTS_DIR / "*.*"))
print(f"[INFO] {split} _dataset: ", len(dataset))

[INFO] loading val dataset
[INFO] val _dataset:  3738


In [30]:
_instance = next(iter(dataset))

In [31]:
type(_instance)

tensorflow.python.framework.ops.EagerTensor

In [32]:
_instance = tf.strings.as_string(_instance).numpy().decode('utf-8')



In [34]:
_instance_path = Path(_instance)

In [35]:
_instance_path

PosixPath('data/VEDAI/val_input/0001_0_1233-9_00000671.png')

In [43]:
tf.constant(str(15) +"" , dtype=tf.string)

<tf.Tensor: shape=(), dtype=string, numpy=b'15_s'>

In [9]:
# dataset2 = dataset.map(lambda x: tf.py_function(_preprocess_instance, [x], [tf.float32, tf.float32, tf.float32, tf.string]))